In [ ]:
import pprint as pp
import pandas as pd
import os
import folium
import folium.plugins
import geopandas as gpd
from shapely.geometry import Point

# Set the working directory
os.chdir('/Users/mikkelroerbaek/Documents/GitHub/graphs')
# filepath = 'C:/Users/mikke/Desktop/trafficdata/US_Accidents_March23.csv'

# # Load the dataset
# accident_data = pd.read_csv(filepath)

# Group by state and get their sizes
# state_sizes = accident_data.groupby('State').size()
# state_sizes.to_csv('analysis_data/state_sizes.csv')




In [ ]:
filepath = 'C:/Users/mikke/Desktop/trafficdata/US_Accidents_March23.csv'

# Load the dataset
accident_data = pd.read_csv(filepath)

county_sizes = accident_data.groupby('County').size()
county_sizes.to_csv('analysis_data/county_sizes.csv')


In [ ]:
# Load the staterenamings.csv file
staterenamings = pd.read_csv('staterenamings.csv')

state_sizes = pd.read_csv('analysis_data/state_sizes.csv', header=None, names=['State', 'count'])

# Merge the state_sizes with the staterenamings
state_sizes = state_sizes.reset_index()
state_sizes = state_sizes.rename(columns={0: 'count'})
state_sizes = state_sizes.merge(staterenamings, left_on='State', right_on='Abbreviation')




In [ ]:
import folium
import pandas as pd

# Create a map centered on the USA
map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add choropleth layer to the map
folium.Choropleth(
    geo_data='us-states.json',  # GeoJSON file containing state boundaries
    name='choropleth',
    data=state_sizes,
    columns=['State_y', 'count'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='State Traffic Accidents'
).add_to(map_usa)

# Display the map
map_usa


In [ ]:
import folium
import pandas as pd

# Load the county_sizes.csv file
county_sizes = pd.read_csv('analysis_data\county_sizes.csv')

county_sizes.rename(columns={'Unnamed: 0': 'count'}, inplace=True)

county_sizes.columns = ['County', 'count']

# Create a map centered on the USA
map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add choropleth layer to the map
folium.Choropleth(
    geo_data='counties.geojson',  # GeoJSON file containing county boundaries
    name='choropleth',
    data=county_sizes,
    columns=['County', 'count'],
    key_on='feature.properties.NAME',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='County Traffic Accidents'
).add_to(map_usa)

# Display the map
map_usa

In [ ]:






# LA_data = accident_data[accident_data['City'] == 'Los Angeles']

# LA_data.to_csv('analysis_data/LA_data.csv')



In [ ]:
import pandas as pd
import folium

LA_data = pd.read_csv('analysis_data/LA_data.csv')

# # Create a map centered on Los Angeles
# map_LA = folium.Map(location=[34.0522, -118.2437], zoom_start=10)
# for index, row in LA_data.iterrows():
#     folium.Marker(
#         location=[row['Start_Lat'], row['Start_Lng']],
#         popup=row['Description'],
#         icon=folium.Icon(color='red', icon='info-sign')
#     ).add_to(map_LA)

# # Save the map as an image file
# map_LA.save('map_LA.png')

# print(LA_data.columns)

# null_proportion = LA_data.isnull().mean() * 100
# print(null_proportion)

# print(LA_data['Station'].value_counts())

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

# LA_data = pd.read_csv('analysis_data/LA_data.csv')
gdf = gpd.read_file(r"/Users/mikkelroerbaek/Documents/GitHub/graphs/data/network['Los Angeles, California, USA'].gpkg", layer='nodes')

# Create the geometry column using the Start_Lat and Start_Lng columns
geometry = [Point(xy) for xy in zip(LA_data['Start_Lng'], LA_data['Start_Lat'])]



# Create the GeoDataFrame
LA_gdf = gpd.GeoDataFrame(LA_data, geometry=geometry)


print(gdf.crs)
LA_gdf.set_crs(epsg=4326, inplace=True)
LA_gdf = LA_gdf.to_crs(epsg=6423)
gdf = gdf.to_crs(epsg=6423)


LA_gdf['geometry'].head()



In [ ]:
import geopandas as gpd

def add_accident_counts(gdf, LA_gdf, range_):
    gdf = gdf.copy()
    LA_gdf = LA_gdf.copy()
    gdf['geometry'] = gdf.geometry.buffer(range_)
    joined = gpd.sjoin(LA_gdf, gdf, how='inner', op='intersects')
    
    def nearest_intersection(accident):
        intersections = joined[joined['ID'] == accident].index_right
        if not intersections.empty:
            nearest = min(intersections, key=lambda intersection: LA_gdf[LA_gdf['ID'] == accident].geometry.distance(gdf.loc[intersection].geometry).min())
            return nearest
        else:
            return None

    LA_gdf['nearest_intersection'] = LA_gdf['ID'].apply(nearest_intersection)
    accident_counts = LA_gdf['nearest_intersection'].value_counts()
    gdf['accident_count'] = gdf.index.map(accident_counts.to_dict())
    gdf['accident_count'] = gdf['accident_count'].fillna(0)
    gdf.to_file(f'analysis_data/intersections_with_accidents_range{range_}.gpkg', driver='GPKG')

    gdf['geometry'] = gdf.apply(lambda row: Point(row['x'], row['y']), axis=1)


range_list = [5]

for i in range_list:
    add_accident_counts(gdf, LA_gdf, i)
    print(f'Range {i} meters done')






In [2]:
import webbrowser
import pickle
import osmnx as ox
import folium   
import networkx as nx
import matplotlib.pyplot as plt


def get_graph_data(place_names, display=False, intersection_focus=False, focus_ignore_edges=False, intersection_edges_quanity=3, maxspeed_fallback=25, basic_stats_toggle=True, interactive_map=False, simple=True, centrality=False, save_data=True, save_data_format='gpkg', features=None):
    
    return_data = {}
    
    if save_data:
        save_storage = {}
    
    if interactive_map:
        # Initialize a storage for later adjusting the map view to fit all places
        all_bounds = []
        
        # Initialize an empty folium map, will be set after the first place is processed
        m = None
    
    for place_name in place_names:
        network_type = "drive"
        
        try:
            G = ox.graph_from_place(place_name, network_type=network_type, simplify=simple)
            if features is not None:
                features_data = ox.features_from_place(place_name, features)
        except ValueError as e:
            print(f"Skipped {place_name}: {e}")
            continue
        
        G_projected = ox.project_graph(G)
        
        # Convert to GeoDataFrame
        gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
        
        # Get area of place
        nodes_proj = ox.graph_to_gdfs(G_projected, edges=False)
        graph_area_m = nodes_proj.unary_union.convex_hull.area

        # Define a fallback speed in km/h (e.g., 30 km/h)
        fallback_speed = maxspeed_fallback

        # Enrich the graph with estimated speed limits and travel times
        # Using a fallback speed for all edges
        G_projected = ox.add_edge_speeds(G_projected, fallback=fallback_speed)  # Adds 'speed_kph' attribute to edges
        G_projected = ox.add_edge_travel_times(G_projected)  # Adds or overwrites the 'travel_time' attribute to edges
        
        # Calculating basic stats
        basic_stats = ox.stats.basic_stats(G_projected)
        
        if basic_stats_toggle:
            if features is None:
                return_data[f'{place_name}'] = {'graph': G, 'basic_stats': basic_stats, 'gdf_nodes': gdf_nodes, 'gdf_edges': gdf_edges}
            else:
                return_data[f'{place_name}'] = {'graph': G, 'basic_stats': basic_stats, 'gdf_nodes': gdf_nodes, 'gdf_edges': gdf_edges, 'features': features_data}
        else:
            if features is None:
                return_data[f'{place_name}'] = {'graph': G, 'gdf_nodes': gdf_nodes, 'gdf_edges': gdf_edges}
            else:
                return_data[f'{place_name}'] = {'graph': G, 'gdf_nodes': gdf_nodes, 'gdf_edges': gdf_edges, 'features': features_data}
            
        # Identifying intersections (nodes with adjusted degree > 2)
        node_degrees = dict(G_projected.degree())
        
        storage = {}
        # Depend on wether or not to focus on intersections
        if intersection_focus == True:
            intersection_nodes = [node for node, degree in node_degrees.items() if degree >= intersection_edges_quanity*2]  # Degree > 4 in bidirectional graph

            # Extracting and printing information about intersections
            for node in intersection_nodes:
                node_data = G_projected.nodes[node]
                
                # Find edges connected to this intersection node
                edges = list(G_projected.edges(node, data=True))
                storage[f'{node}'] = ([node_data])
                storage[f'{node}'].append(edges)
        else:
            # Extracting and printing information about intersections
            for node in node_degrees:           
                intersection_node = False
                degree = node_degrees[node] 
                if degree >= intersection_edges_quanity*2:
                    intersection_node = True
                
                node_data = G_projected.nodes[node]
                
                # Find edges connected to this intersection node
                edges = list(G_projected.edges(node, data=True))
                
                storage[f'{node}'] = ([node_data])
                storage[f'{node}'].append(edges)
                storage[f'{node}'][0]['intersection_node'] = intersection_node
        
        if save_data:
            save_storage[f'{place_name}'] = storage
        
        if display:
            if interactive_map == False:
                if centrality == True:
                    edge_centrality = nx.closeness_centrality(nx.line_graph(G))
                    nx.set_edge_attributes(G, edge_centrality, "edge_centrality")
                    ec = ox.plot.get_edge_colors_by_attr(G, "edge_centrality", cmap="inferno")
                    fig, ax = ox.plot_graph(G, edge_color=ec, edge_linewidth=2, node_size=0)
                else:
                    fig, ax = plt.subplots()
                    
                    # Plot the edges on the axes
                    gdf_edges.plot(ax=ax, linewidth=1, edgecolor='#BC8F8F')

                    # Plot the nodes on the same axes
                    gdf_nodes.plot(ax=ax, markersize=10, color='red')
                    
                    plt.show()
            else:
                nodes_gdf, edges_gdf = ox.graph_to_gdfs(G, nodes=True, edges=True)
                
                if m is None:
                    # Find map center from the first place's bounds to initialize map
                    bounds = gdf_nodes.total_bounds
                    center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]
                    m = folium.Map(location=center, zoom_start=14)
                all_bounds.append(gdf_nodes.total_bounds)
                
                # Add edges with detailed geometies and popups for speed from G_projected
                for u, v, key, data in G_projected.edges(keys=True, data=True):
                    if focus_ignore_edges != True and intersection_focus == True:
                        if v in intersection_nodes:
                            info_parts = [
                                f"<strong>ID:</strong> {data.get('osmid', 'N/A')}",
                                f"<strong>Name:</strong> {data.get('name', 'N/A')}",
                                f"<strong>Speed:</strong> {data.get('speed_kph', 'N/A')} km/h",
                                f"<strong>Length:</strong> {data.get('length', 'N/A')} meters",
                                f"<strong>Highway:</strong> {data.get('highway', 'N/A')}",
                                f"<strong>Oneway:</strong> {data.get('oneway', 'N/A')}",
                                f"<strong>Reversed:</strong> {data.get('reversed', 'N/A')}",
                                f"<strong>Travel time:</strong> {data.get('travel_time', 'N/A')} seconds",
                                f"<strong>Connected Nodes:</strong> {u}, {v}",
                            ]
                            popup_text = '<br>'.join([f"<span style='font-size: 12pt;'>{part}</span>" for part in info_parts])
                            popup = folium.Popup(html=popup_text, max_width=250)
                            if (u, v, key) in edges_gdf.index:
                                geom = [(lat, lon) for lon, lat in edges_gdf.loc[(u, v, key), 'geometry'].coords]
                                folium.PolyLine(
                                    locations=geom,
                                    color="blue",
                                    weight=7,  # Adjust weight as needed
                                    popup=popup
                                ).add_to(m)
                    else:
                        info_parts = [
                            f"<strong>ID:</strong> {data.get('osmid', 'N/A')}",
                            f"<strong>Name:</strong> {data.get('name', 'N/A')}",
                            f"<strong>Speed:</strong> {data.get('speed_kph', 'N/A')} km/h",
                            f"<strong>Length:</strong> {data.get('length', 'N/A')} meters",
                            f"<strong>Highway:</strong> {data.get('highway', 'N/A')}",
                            f"<strong>Oneway:</strong> {data.get('oneway', 'N/A')}",
                            f"<strong>Reversed:</strong> {data.get('reversed', 'N/A')}",
                            f"<strong>Travel time:</strong> {data.get('travel_time', 'N/A')} seconds",
                            f"<strong>Connected Nodes:</strong> {u}, {v}",
                        ]
                        popup_text = '<br>'.join([f"<span style='font-size: 12pt;'>{part}</span>" for part in info_parts])
                        popup = folium.Popup(html=popup_text, max_width=250)
                        if (u, v, key) in edges_gdf.index:
                            geom = [(lat, lon) for lon, lat in edges_gdf.loc[(u, v, key), 'geometry'].coords]
                            folium.PolyLine(
                                locations=geom,
                                color="blue",
                                weight=7,  # Adjust weight as needed
                                popup=popup
                            ).add_to(m)
                
                for node, data in nodes_gdf.iterrows():
                    if intersection_focus == True:
                        if node in intersection_nodes:
                            degree = node_degrees[node]
                            node_color = 'lime' if degree >= intersection_edges_quanity*2 else 'red'
                            node_attrs = storage[str(node)][0]
                            edge_tuples = storage[str(node)][1]
                            osmid_list = [str(edge[2]['osmid']) for edge in edge_tuples]  # Adjust based on actual structure
                            osmids_list_str = ', '.join(osmid_list)
                            info_parts = [
                                f"<strong>ID:</strong> {node}",
                                f"<strong>Y, X:</strong> {data['y']}, {data['x']}",
                                f"<strong>Lat, Lon:</strong> {data['y']}, {data['x']}",
                                f"<strong>Connected Edges OSMID:</strong> {osmids_list_str}",
                            ]
                            popup_text = '<br>'.join([f"<span style='font-size: 12pt;'>{part}</span>" for part in info_parts])
                            popup = folium.Popup(html=popup_text, max_width=300)
                            folium.CircleMarker(
                                location=(data['y'], data['x']),
                                radius=8,  # Visibility adjustment
                                color=node_color,  # Brighter green
                                fill=True,
                                fill_color="lime",
                                popup=popup
                            ).add_to(m)
                    else:
                        if node in node_degrees:
                            degree = node_degrees[node]
                            node_color = 'lime' if degree >= intersection_edges_quanity*2 else 'red'
                            node_attrs = storage[str(node)][0]
                            edge_tuples = storage[str(node)][1]
                            osmid_list = [str(edge[2]['osmid']) for edge in edge_tuples]  # Adjust based on actual structure
                            osmids_list_str = ', '.join(osmid_list)
                            info_parts = [
                                f"<strong>ID:</strong> {node}",
                                f"<strong>Y, X:</strong> {data['y']}, {data['x']}",
                                f"<strong>Lat, Lon:</strong> {data['y']}, {data['x']}",
                                f"<strong>Connected Edges OSMID:</strong> {osmids_list_str}",
                            ]
                            popup_text = '<br>'.join([f"<span style='font-size: 12pt;'>{part}</span>" for part in info_parts])
                            popup = folium.Popup(html=popup_text, max_width=300)
                            folium.CircleMarker(
                                location=(data['y'], data['x']),
                                radius=8,  # Visibility adjustment
                                color=node_color,  # Brighter green
                                fill=True,
                                fill_color="lime",
                                popup=popup
                            ).add_to(m)
        if save_data:
            if save_data_format == 'gpkg':
                # save street network as GeoPackage to work with in GIS
                ox.save_graph_geopackage(G, filepath=f'./data/network{place_name}.gpkg')
            if save_data_format == 'graphml':
                # save street network as GraphML file to work with later in OSMnx or networkx or gephi
                ox.save_graphml(G, filepath=f'./data/network{place_name}.graphml')
    
    if save_data and save_data_format == 'dict':
        with open(f'./dictData/{place_name}.pickle', 'wb') as handle:
            pickle.dump(return_data, handle)
    
    if interactive_map and m is not None:
        # Adjust the map to fit all places
        bounds = [min([b[0] for b in all_bounds]), min([b[1] for b in all_bounds]),
                  max([b[2] for b in all_bounds]), max([b[3] for b in all_bounds])]
        m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
        
        folium.LayerControl().add_to(m)
        
        map_filename = 'map.html'
        m.save(map_filename)
        webbrowser.open(map_filename, new=2)
    
    return return_data
    

places = ['Los Angeles, California, USA']

graph_data = get_graph_data(place_names=places, display=False, intersection_focus=False, focus_ignore_edges=False, intersection_edges_quanity=3, maxspeed_fallback=30, basic_stats_toggle=True, interactive_map=False, simple=True, centrality=False, save_data=False, save_data_format='gpkg', features={'amenity': ['kindergarten','school', 'bar', 'fast_food', 'pub', 'college', 'library', 'university', 'courthouse', 'police', 'prison', 'hospital'], 'public_transport': 'stop_position'})
 


edges = graph_data['Los Angeles, California, USA']['gdf_edges']

edges_with_linestrings = edges

print(edges_with_linestrings['geometry'].head())

# edges[edges['osmid'] == 1249289296]

# for i in edges['geometry']:
#     value = i.wkt  # Assuming 'i' is the LineString variable
#     if 'LINESTRING' in value:
#         print("The value includes 'LINESTRING'")
#     else:
#         print("The value does not include 'LINESTRING'")

 

u       v           key
653688  21300195    0      LINESTRING (-118.42955 34.02703, -118.42948 34...
        1614655105  0      LINESTRING (-118.42955 34.02703, -118.42980 34...
653689  1711811908  0      LINESTRING (-118.42547 34.02864, -118.42538 34...
        122734100   0      LINESTRING (-118.42547 34.02864, -118.42556 34...
        1711811906  0      LINESTRING (-118.42547 34.02864, -118.42554 34...
Name: geometry, dtype: geometry


In [3]:
import pandas as pd
features = graph_data['Los Angeles, California, USA']['features']

features['feature'] = features['public_transport'].fillna('') + features['amenity'].fillna('')

features = features[['feature', 'geometry']]

# Categorize the feature column as new columns with True or False values
feature_dummies = pd.get_dummies(features['feature'])

# Concatenate the feature dummies with the original GeoDataFrame
features_with_dummies = pd.concat([features, feature_dummies], axis=1)

print(features_with_dummies.columns)

geometry = features_with_dummies['geometry']

# Display the updated GeoDataFrame
features_with_dummies.rename(columns={'stop_position': 'bus_stop'}, inplace=True)

features_with_dummies = gpd.GeoDataFrame(features_with_dummies, geometry=geometry)

features_with_dummies.to_crs(6423, inplace=True)

features_with_dummies.drop(columns=['feature'], inplace=True)

Index(['feature', 'geometry', 'bar', 'college', 'courthouse', 'fast_food',
       'hospital', 'kindergarten', 'library', 'police', 'prison', 'pub',
       'school', 'stop_position', 'university'],
      dtype='object')


In [4]:
import geopandas as gpd
from shapely.geometry import Point

nodes = gpd.read_file("/Users/mikkelroerbaek/Documents/GitHub/graphs/analysis_data/intersections_with_accidents_range5.gpkg")
nodes = gpd.GeoDataFrame(nodes, geometry='geometry')

print(feature_dummies.shape)
print(nodes.shape)

#feature collection range, 10 meters
data_range = 50

# Buffer the nodes so they can find features close by
nodes['geometry'] = nodes.buffer(data_range)

joined_data = gpd.sjoin(nodes, features_with_dummies, how='left', op='intersects')

# return the geometry data to the original datapoint
joined_data['geometry'] = joined_data.apply(lambda row: Point(row['x'], row['y']), axis=1)
nodes['geometry'] = nodes.apply(lambda row: Point(row['x'], row['y']), axis=1)

print(joined_data.columns)

# joined_data.drop(['index_right0', 'index_right1'], axis=1, inplace=True)

print(joined_data[['library', 'police', 'prison', 'pub', 'school', 'bus_stop', 'university']])


selected_features = ['bar', 'college', 'courthouse', 'fast_food', 'hospital', 'kindergarten', 'library', 'police', 'prison', 'pub', 'school', 'bus_stop', 'university']



filtered_features = joined_data.loc[:, selected_features]

for column in filtered_features.columns:
    # Convert the column to boolean, treating NaN values as False
    bool_column = filtered_features[column].fillna(False).astype(bool)
    
    true_count = bool_column.sum()  # True is treated as 1, so summing gives the count of True values
    false_count = (~bool_column).sum()  # False is treated as 0, so inverting and summing gives the count of False values
    nan_count = filtered_features[column].isna().sum()  # isna gives True for NaN values, so summing gives the count of NaN values
    
    print(f"Count of True values in {column}: {true_count}")
    print(f"Count of False values in {column}: {false_count}")
    print(f"Count of NaN values in {column}: {nan_count}")
    print()




joined_data.head()

joined_data.drop(['index_right0', 'index_right1'], axis=1, inplace=True)



(3265, 13)
(49446, 8)


/Users/mikkelroerbaek/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Index(['osmid', 'y', 'x', 'highway', 'street_count', 'ref', 'accident_count',
       'geometry', 'index_right0', 'index_right1', 'bar', 'college',
       'courthouse', 'fast_food', 'hospital', 'kindergarten', 'library',
       'police', 'prison', 'pub', 'school', 'bus_stop', 'university'],
      dtype='object')
      library police prison    pub school bus_stop university
0       False  False  False  False  False    False      False
1         NaN    NaN    NaN    NaN    NaN      NaN        NaN
2         NaN    NaN    NaN    NaN    NaN      NaN        NaN
3       False  False  False  False  False    False      False
4         NaN    NaN    NaN    NaN    NaN      NaN        NaN
...       ...    ...    ...    ...    ...      ...        ...
49441     NaN    NaN    NaN    NaN    NaN      NaN        NaN
49442     NaN    NaN    NaN    NaN    NaN      NaN        NaN
49443     NaN    NaN    NaN    NaN    NaN      NaN        NaN
49444     NaN    NaN    NaN    NaN    NaN      NaN        NaN
49445

/var/folders/3w/58c0h_m95x9g00pqt9frwfgw0000gn/T/ipykernel_9579/977719080.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bool_column = filtered_features[column].fillna(False).astype(bool)
/var/folders/3w/58c0h_m95x9g00pqt9frwfgw0000gn/T/ipykernel_9579/977719080.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bool_column = filtered_features[column].fillna(False).astype(bool)
/var/folders/3w/58c0h_m95x9g00pqt9frwfgw0000gn/T/ipykernel_9579/977719080.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and

In [5]:
import pandas as pd
joined_data[selected_features] = joined_data[selected_features].fillna(False).astype(bool)


# for column in joined_data.columns:

#     if column in selected_features:
        
#         true_count = bool_column.sum()  # True is treated as 1, so summing gives the count of True values
#         false_count = (~bool_column).sum()  # False is treated as 0, so inverting and summing gives the count of False values
#         nan_count = joined_data[column].isna().sum()  # isna gives True for NaN values, so summing gives the count of NaN values
        
#         print(f"Count of True values in {column}: {true_count}")
#         print(f"Count of False values in {column}: {false_count}")
#         print(f"Count of NaN values in {column}: {nan_count}")
#         print()





# joined_data[joined_data['osmid'] == 11611743521]


edges_with_linestrings.rename(columns={'geometry': 'geometry_linestring'}, inplace=True)


first_index_value = edges_with_linestrings.index.get_level_values(0)[0]
geometry_linestrings = edges_with_linestrings['geometry_linestring']



# Create a dataframe
df = pd.DataFrame({'geometry_linestring': edges_with_linestrings['geometry_linestring'],
                   'osmid': edges_with_linestrings.index.get_level_values(0)})

df.reset_index(inplace=True)


# df.drop(['u', 'v', 'key'], axis=1, inplace=True)


df.rename(columns={'u': 'from_node', 'v': 'to_node'}, inplace=True)


file_path = "/Users/mikkelroerbaek/Documents/GitHub/graphs/edges_for_Los Angeles, California, USA"
edges_data = pd.read_csv(file_path, sep=';')

merged_df = df.merge(edges_data, on=['from_node', 'to_node'])


pd.set_option('display.max_colwidth', None)

merged_df.drop_duplicates(inplace=True)


merged_df.drop('osmid', axis=1, inplace=True)
merged_df.drop('Unnamed: 0', axis=1, inplace=True)

final_df = pd.merge(joined_data, merged_df, left_on='osmid', right_on='from_node')

final_df.drop_duplicates(subset=['osmid', 'to_node'], inplace=True)


final_df


/var/folders/3w/58c0h_m95x9g00pqt9frwfgw0000gn/T/ipykernel_9579/363926139.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  joined_data[selected_features] = joined_data[selected_features].fillna(False).astype(bool)


,osmid,y,x,highway,street_count,ref,accident_count,geometry,bar,college,...,school,bus_stop,university,from_node,to_node,key,geometry_linestring,road_id,road_length,max_speed
0,653688,34.027025,-118.429548,traffic_signals,3,,0.0,POINT (-118.430 34.027),False,False,...,False,False,False,653688,21300195,0,"LINESTRING (-118.42955 34.02703, -118.42948 34.02705, -118.42925 34.02714, -118.42910 34.02719, -118.42879 34.02731, -118.42867 34.02736)","[907145138, 895315641, 895315642, 398770659]",88.746,56.3
1,653688,34.027025,-118.429548,traffic_signals,3,,0.0,POINT (-118.430 34.027),False,False,...,False,False,False,653688,1614655105,0,"LINESTRING (-118.42955 34.02703, -118.42980 34.02693, -118.43029 34.02673, -118.43055 34.02663)","[398770658, 759468526, 759468527]",102.260,56.3
2,653689,34.028639,-118.425473,traffic_signals,4,,0.0,POINT (-118.425 34.029),False,False,...,False,False,False,653689,1711811908,0,"LINESTRING (-118.42547 34.02864, -118.42538 34.02867, -118.42533 34.02869, -118.42528 34.02871)",398771138,19.377,56.3
3,653689,34.028639,-118.425473,traffic_signals,4,,0.0,POINT (-118.425 34.029),False,False,...,False,False,False,653689,122734100,0,"LINESTRING (-118.42547 34.02864, -118.42556 34.02860, -118.42561 34.02858, -118.42572 34.02853, -118.42655 34.02821)","[404964730, 398771139]",109.729,56.3
4,653689,34.028639,-118.425473,traffic_signals,4,,0.0,POINT (-118.425 34.029),False,False,...,False,False,False,653689,1711811906,0,"LINESTRING (-118.42547 34.02864, -118.42554 34.02872, -118.42558 34.02875, -118.42558 34.02876, -118.42560 34.02878)",643327598,19.494,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139089,11754616090,34.117880,-118.292772,,3,,0.0,POINT (-118.293 34.118),False,False,...,False,False,False,11754616090,635572488,0,"LINESTRING (-118.29277 34.11788, -118.29285 34.11789, -118.29334 34.11792, -118.29359 34.11796, -118.29373 34.11800, -118.29384 34.11806, -118.29396 34.11810)","[1083742289, 1265414917]",114.194,34.1
139090,11754895962,33.966683,-118.291880,,3,,0.0,POINT (-118.292 33.967),False,False,...,False,False,False,11754895962,1414239953,0,"LINESTRING (-118.29188 33.96668, -118.29175 33.96668)",13295327,11.832,40.4
139091,11754895962,33.966683,-118.291880,,3,,0.0,POINT (-118.292 33.967),False,False,...,False,False,False,11754895962,1414239676,0,"LINESTRING (-118.29188 33.96668, -118.29187 33.96590, -118.29187 33.96572, -118.29185 33.96567, -118.29181 33.96562, -118.29175 33.96557)",127864996,127.175,34.1
139092,11756453980,34.099293,-118.315840,,3,,1.0,POINT (-118.316 34.099),False,False,...,False,False,False,11756453980,1840698844,0,"LINESTRING (-118.31584 34.09929, -118.31593 34.09929, -118.31657 34.09929)",173221034,67.515,40.4


In [7]:
import numpy as np
import math
from shapely.geometry import LineString

def process_osmids(df):
    def extract_vector(edge):
        return (edge.coords[1][0] - edge.coords[0][0], edge.coords[1][1] - edge.coords[0][1])

    def calculate_angle_between(v1, v2):
        angle1 = math.atan2(v1[1], v1[0])
        angle2 = math.atan2(v2[1], v2[0])
        return math.degrees((angle2 - angle1) % (2 * math.pi))

    df['angle'] = None
    df['to_node_pairs'] = None
    df['opposite_angle'] = None
    df['to_node_pairs_opposite'] = None
    df['adjacent_max_speed'] = None
    df['adjacent_road_length'] = None
    df['opposite_adjecent_max_speed'] = None
    df['opposite_adjecent_road_length'] = None

    to_node_dict = df.set_index('geometry_linestring')['to_node'].to_dict()
    
    for osmid in df['osmid'].unique():
        osmid_df = df[df['osmid'] == osmid]
        edges_with_indexes = [(index, row['geometry_linestring']) for index, row in osmid_df.iterrows()]
        vectors_with_angles = sorted([(extract_vector(edge), index) for index, edge in edges_with_indexes], key=lambda x: math.atan2(x[0][1], x[0][0]))
        count_osmid = df[df['osmid'] == osmid].shape[0]
        
        if len(edges_with_indexes) > 0:
            common_start_node = osmid_df.loc[edges_with_indexes[0][0]]['from_node']

            for i in range(len(vectors_with_angles)):
                v1, index1 = vectors_with_angles[i]
                v2, index2 = vectors_with_angles[(i + 1) % len(vectors_with_angles)]
                v3, index3 = vectors_with_angles[(i - 1) % len(vectors_with_angles)]
                angle = calculate_angle_between(v1, v2)
                angle2 = calculate_angle_between(v3, v1) 
                
                
                current_to_node = to_node_dict[df.loc[index1, 'geometry_linestring']]
                next_to_node = to_node_dict[df.loc[index2, 'geometry_linestring']]
                previous_to_node = to_node_dict[df.loc[index3, 'geometry_linestring']]
                next_to_speed = df.loc[index2, 'max_speed']
                previous_to_speed = df.loc[index3, 'max_speed']
                next_to_length = df.loc[index2, 'road_length']
                previous_to_length = df.loc[index3, 'road_length']
                
                df.loc[index1, ['street_count_computed','angle', 'to_node_pairs', 'opposite_angle', 'to_node_pairs_opposite', 'adjacent_max_speed', 'adjacent_road_length', 'opposite_adjecent_max_speed', 'opposite_adjecent_road_length']] = count_osmid ,angle, str([current_to_node, next_to_node]), angle2, str([current_to_node, previous_to_node]), next_to_speed, next_to_length, previous_to_speed, previous_to_length

    return df

# Use the function on your DataFrame
final_df = process_osmids(final_df)

In [9]:
filtered_rows = final_df[final_df['angle'].notnull()]


# filtered_rows.drop(['geometry_linestring', 'from_node', 'to_node', 'x', 'y', 'geometry', 'road_id', 'to_node_pairs', 'to_node_pairs_opposite', 'key', 'ref'], axis=1, inplace=True)


# filtered_rows.rename(columns={'highway': 'intersection_attribute'}, inplace=True)


filtered_rows.to_csv('analysis_data/nodes_with_angles.csv')

filtered_rows




,osmid,y,x,highway,street_count,ref,accident_count,geometry,bar,college,...,max_speed,angle,to_node_pairs,opposite_angle,to_node_pairs_opposite,adjacent_max_speed,adjacent_road_length,opposite_adjecent_max_speed,opposite_adjecent_road_length,street_count_computed
0,653688,34.027025,-118.429548,traffic_signals,3,,0.0,POINT (-118.430 34.027),False,False,...,56.3,179.193465,"[21300195, 1614655105]",180.806535,"[21300195, 1614655105]",56.3,102.26,56.3,102.26,2.0
1,653688,34.027025,-118.429548,traffic_signals,3,,0.0,POINT (-118.430 34.027),False,False,...,56.3,180.806535,"[1614655105, 21300195]",179.193465,"[1614655105, 21300195]",56.3,88.746,56.3,88.746,2.0
2,653689,34.028639,-118.425473,traffic_signals,4,,0.0,POINT (-118.425 34.029),False,False,...,56.3,111.110878,"[1711811908, 1711811906]",68.970675,"[1711811908, 1615552822]",47.0,19.494,40.4,99.592,4.0
3,653689,34.028639,-118.425473,traffic_signals,4,,0.0,POINT (-118.425 34.029),False,False,...,56.3,108.675482,"[122734100, 1615552822]",71.242965,"[122734100, 1711811906]",40.4,99.592,47.0,19.494,4.0
4,653689,34.028639,-118.425473,traffic_signals,4,,0.0,POINT (-118.425 34.029),False,False,...,47.0,71.242965,"[1711811906, 122734100]",111.110878,"[1711811906, 1711811908]",56.3,109.729,56.3,19.377,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139089,11754616090,34.117880,-118.292772,,3,,0.0,POINT (-118.293 34.118),False,False,...,34.1,92.088906,"[635572488, 11754616089]",267.911094,"[635572488, 11754616089]",40.4,19.921,40.4,19.921,2.0
139090,11754895962,33.966683,-118.291880,,3,,0.0,POINT (-118.292 33.967),False,False,...,40.4,270.565232,"[1414239953, 1414239676]",89.434768,"[1414239953, 1414239676]",34.1,127.175,34.1,127.175,2.0
139091,11754895962,33.966683,-118.291880,,3,,0.0,POINT (-118.292 33.967),False,False,...,34.1,89.434768,"[1414239676, 1414239953]",270.565232,"[1414239676, 1414239953]",40.4,11.832,40.4,11.832,2.0
139092,11756453980,34.099293,-118.315840,,3,,1.0,POINT (-118.316 34.099),False,False,...,40.4,90.849317,"[1840698844, 1079419371]",269.150683,"[1840698844, 1079419371]",57.6,21.601,57.6,21.601,2.0


In [ ]:
import folium

osmid = 653688

# Filter the dataframe based on the specific osmid
filtered_df = final_df[final_df['osmid'] == osmid]

print(type(filtered_df))
print(filtered_df)
first_row = filtered_df.iloc[0]
geometry_value = first_row['geometry']

x, y = geometry_value.coords[0]

# Create a map centered around the 'geometry' point
m = folium.Map(location=[y, x], zoom_start=12)

# Add each line from the filtered dataframe to the map
for index, row in filtered_df.iterrows():
    line = row['geometry_linestring']
    folium.PolyLine(
        locations=[(y, x) for x, y in line.coords], 
        color='blue',
        tooltip=f" Node: {row['to_node']} Angle: {row['angle']}, To Node Pairs: {row['to_node_pairs']}, Opposite Angle: {row['opposite_angle']}, To Node Pairs Opposite: {row['to_node_pairs_opposite']}"
    ).add_to(m)

# Display the map
m